In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")  # 加载数据
checkpoint = "bert-base-uncased"  
tokenizer = AutoTokenizer.from_pretrained(checkpoint)  # 加载分词器

# 定义分词函数，超长截断
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)  # 批量分词
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # 定义数据收集器，用于将数据填充到同一长度

## 训练前准备

### 迭代批次的数据加载

In [ ]:
# 删除不需要的列
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
# 重命名label列为labels
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# 设置格式为torch，以便后续使用pytorch训练
tokenized_datasets.set_format("torch")
# 查看数据集
tokenized_datasets["train"].column_names

["attention_mask", "input_ids", "labels", "token_type_ids"]

定义数据加载器

In [ ]:
from torch.utils.data import DataLoader

# 定义训练集和验证集的dataloader
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)  # shuffle=True表示打乱数据，batch_size=8表示每个batch的样本数为8，collate_fn=data_collator表示使用数据收集器，将数据填充到同一长度
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

可以通过一下代码检验数据加载器是否正确

In [ ]:
# 查看一个batch的数据
# for batch in train_dataloader:
#     break
# {k: v.shape for k, v in batch.items()}

{'attention_mask': torch.Size([8, 65]),
 'input_ids': torch.Size([8, 65]),
 'labels': torch.Size([8]),
 'token_type_ids': torch.Size([8, 65])}

### 开始训练

In [ ]:
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler
import torch
from tqdm import tqdm

# 加载模型, 并设置为二分类
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
# 定义优化器，使用AdamW，学习率为3e-5的线性学习率衰减
optimizer = AdamW(model.parameters(), lr=3e-5)

# 如果有GPU资源，将模型和数据放到GPU上
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# 定义学习率衰减策略
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)  # 总的训练步数
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# 设置训练进度条
progress_bar = tqdm(range(num_training_steps))

# 开始训练
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

### 评估循环

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])  # 累计所有batch的预测结果和真实标签

metric.compute()  # 计算最终的评估指标

## 使用Accelerate加速训练

In [ ]:
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler
import torch
from tqdm import tqdm



accelerator = Accelerator()






# 加载模型, 并设置为二分类
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2)
# 定义优化器，使用AdamW，学习率为3e-5的线性学习率衰减
optimizer = AdamW(model.parameters(), lr=3e-5)






# 如果有GPU资源，将模型和数据放到GPU上
# device = torch.device(
#     "cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)

train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(train_dataloader, eval_dataloader, model, optimizer)






# 定义学习率衰减策略
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)  # 总的训练步数
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# 设置训练进度条
progress_bar = tqdm(range(num_training_steps))

# 开始训练
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        # batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        # loss.backward()
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

## 进行分布式训练
- accelerate config：配置分布式训练环境
- accelerate launch train.py：启动分布式训练

In [ ]:
# 在notebook中使用，需要将代码粘贴到training_function函数中
from accelerate import notebook_launcher

notebook_launcher(training_function)